# **PART 3 Extract MD&A Part**

Get the MD&A part for firms on a specific date (given) **Based on the full content we've already had**

RETURN a dataframe(MDA_10k) with one more column (10KText_MDA)
    
original text (no lowercase check)

1. cik number
2. company name
3. form id (should be 10-K)
4. date
5. file_url
6. 10KText_MDA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re

In [ ]:
import os
import pandas as pd
path = "/content/drive/Shared drives/AFP/Web Scrapping/2019"
os.chdir(path)

document_df_10k = pd.read_csv("FullContent_10k_2019.csv")

## **Extract MDA part - Method 2**

Using regular expression

**!!!**

1. No.22: 1082027 -> Item 6. Management 
2. No.126: 1448038 -> Item 7 –
3. No.137: 1485074 -> Item 7 -
4. No.247: 1724009 -> Item 7. Trustees'
5. No.269: 34956 -> Item 7 —

## Version 2

In [ ]:
def Get_10k_MDAContent(document_fulltext_10k):
    document_fulltext_10k = document_fulltext_10k.lower()

    ## assuming most firms use "item 7."
    startMatch = re.findall('(item\s*7\.?\:?\s*[m])', document_fulltext_10k)
    if len(startMatch) == 0:
        ## some firms use "item 7-"
        startMatch = re.finditer('(item\s*7\s*\–?\-?\—?\s*[m])', document_fulltext_10k)
    else:
        startMatch = re.finditer('(item\s*7\.?\:?\s*[m])', document_fulltext_10k)
    startIdxList = [i.start() for i in startMatch]

    if len(startIdxList) == 0:
        ## some firms use "item 6"
        startMatch = re.findall('(item\s*6\.?\:?\s*[m])', document_fulltext_10k)
        if len(startMatch) == 0:
            startMatch = re.finditer('(item\s*6\s*\–?\-?\—?\s*[m])', document_fulltext_10k)
        else:
            startMatch = re.finditer('(item\s*6\.?\:?\s*[m])', document_fulltext_10k)

        endMatch = re.finditer('(item\s*7\.?\:?\s*\–?\-?\—?\s*)', document_fulltext_10k)
        startIdxList = [i.start() for i in startMatch]
        # endIdxList = [i.start() for i in endMatch]
    else:    
        endMatch = re.finditer('(item\s*7a\.?\:?\s*\–?\-?\—?\s*q)', document_fulltext_10k)
        endIdxList = [i.start() for i in endMatch]
        if len(endIdxList) == 0:
            endMatch = re.finditer('(item\s*8\.?\:?\s*\–?\-?\—?\s*)', document_fulltext_10k)
            endIdxList = [i.start() for i in endMatch]

    startIdx = startIdxList[-1] 
    ## find the first "item 7a" or whatever, just following MDA part
    for idx in endIdxList:
        if idx > startIdx:
            endIdx = idx

    # print((startIdx, endIdx)) # -1 -> not found

    return document_fulltext_10k[startIdx:endIdx], startIdx, endIdx

## Final Application

In [ ]:
MDA_df_10k = document_df_10k[['cik number', 'company name', 'form id', 'date', 'file_url']].copy()
MDA_df_10k.head()

In [ ]:
MDA_df_10k['10KText_MDA'] = None
MDA_df_10k['startIdx'] = None
MDA_df_10k['endIdx'] = None
MDA_df_10k.head()

In [ ]:
ErrorFirm = {}
for i in range(len(document_df_10k['10KText_All'])):
    print(i,  "-----", document_df_10k['company name'][i])
    try:
        MDAcontent, startIdx, endIdx = Get_10k_MDAContent(document_df_10k['10KText_All'][i])
        MDA_df_10k['10KText_MDA'][i] = MDAcontent
        MDA_df_10k['startIdx'][i] = startIdx
        MDA_df_10k['endIdx'][i] = endIdx
    except:
        ## if MDA_df_10k has None, which means the Get_10k_MDAContent doesn't work for that company
        ErrorFirm[i] = document_df_10k['company name'][i]


In [ ]:
MDA_df_10k['Length'] = MDA_df_10k['endIdx'] - MDA_df_10k['startIdx']

In [ ]:
MDA_df_10k_1 = MDA_df_10k[MDA_df_10k["Length"] > 1000].dropna()
MDA_df_10k_1.reset_index()

In [ ]:
MDA_df_10k_1.to_csv("MDA_10k_2019.csv")

Merge

In [ ]:
path_new = "/content/drive/Shared drives/AFP/Web Scrapping"
MDA_3yr = None
for yr in ["2017", "2018", "2019"]:
    MDA_1yr = pd.read_csv(os.path.join(path_new, yr, "MDA_10k_"+yr+".csv"))
    MDA_3yr = pd.concat([MDA_3yr, MDA_1yr])

MDA_3yr.reset_index(inplace=True, drop=True)

In [ ]:
MDA_3yr['Length'] = MDA_3yr['endIdx'] - MDA_3yr['startIdx']

In [ ]:
MDA_3yr.to_csv("MDA_All_3yr.csv")